In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34163 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [5]:
%%time
lonW = 150-360
lonE = 180-360
latL = -32
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym32_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
interp(yt_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym32_1_mod_month)

temp_ym32_1_mod_month.load()
print(temp_ym32_1_mod_month)

xt_ocean = temp_ym32_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym32_1_mod_month = temp_ym32_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym32_1_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym32_1_mod_month = temp_ym32_1_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym32_1_mod_month)

temp_ym32_1_mod = temp_ym32_1_mod_month.groupby('time.year').mean('time')
print(temp_ym32_1_mod)

temp_ym32_1_mod = temp_ym32_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_ym32_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 30)>
dask.array<sub, shape=(300, 35, 30), dtype=float32, chunksize=(1, 25, 30), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -209.5 -208.5 -207.5 ... -182.5 -181.5 -180.5
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
    yt_ocean  int64 -32
<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 30)>
array([[[        nan,         nan,         nan, ..., 20.02348596,
         19.8360281 , 19.67995926],
        [        nan,         nan,         nan, ..., 20.00840855,
         19.81835498, 19.66225639],
        [        nan,         nan,         nan, ..., 19.99186764,
         19.79775418, 19.64154894],
        ...,
        [        nan,         nan,         nan, ...,  2.62143737,
          2.63077187,  2.63212381],
        [        nan,         nan,         nan, ...,  2.34054319,
          2.347413

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
year = temp_ym32_1_mod.year
depth_1_mod = temp_ym32_1_mod.st_ocean
lon_1_mod = temp_ym32_1_mod.xt_ocean

temp_ym32_1_trans_mod = temp_ym32_1_mod
print(np.shape(temp_ym32_1_trans_mod))
temp_ym32_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym32_1')
temp_ym32_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym32_1')
temp_ym32_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym32_1')
for iid, i in enumerate(depth_1_mod):
    for jid, j in enumerate(lon_1_mod):
        temp_ym32_1_slope_mod[iid, jid], _, _, temp_ym32_1_p_value_mod[iid, jid], temp_ym32_1_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym32_1_trans_mod[:, iid, jid])
    print('depth_1_mod ' + str(np.array(i)))
print(temp_ym32_1_slope_mod)

(14, 35, 30)
depth_1_mod -1.15174989182586
depth_1_mod -3.6486741197120196
depth_1_mod -6.564918471676153
depth_1_mod -9.970869194872916
depth_1_mod -13.948708373749158
depth_1_mod -18.594379460593196
depth_1_mod -24.019873617545198
depth_1_mod -30.355885242559452
depth_1_mod -37.754889932903495
depth_1_mod -46.39470194845016
depth_1_mod -56.48256969937929
depth_1_mod -68.25986482598941
depth_1_mod -82.00740970018322
depth_1_mod -98.05146439730592
depth_1_mod -116.77034928747688
depth_1_mod -138.60160118643546
depth_1_mod -164.04943216631892
depth_1_mod -193.69205664349184
depth_1_mod -228.1881444592732
depth_1_mod -268.2812141854562
depth_1_mod -314.800181990729
depth_1_mod -368.65354740782504
depth_1_mod -430.8139438640983
depth_1_mod -502.28929308570173
depth_1_mod -584.0771003035616
depth_1_mod -677.1002526914099
depth_1_mod -782.1267221440287
depth_1_mod -899.6817765561261
depth_1_mod -1029.9681126743835
depth_1_mod -1172.813371535859
depth_1_mod -1327.6620295174746
depth_1_mod -1

In [ ]:
%%time
lonW = 150-360
lonE = 182-360
latL = -32
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym32_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
interp(yt_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym32_025_mod_month)

temp_ym32_025_mod_month.load()
print(temp_ym32_025_mod_month)

xt_ocean = temp_ym32_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym32_025_mod_month = temp_ym32_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym32_025_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym32_025_mod_month = temp_ym32_025_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym32_025_mod_month)

temp_ym32_025_mod = temp_ym32_025_mod_month.groupby('time.year').mean('time')
print(temp_ym32_025_mod)

temp_ym32_025_mod = temp_ym32_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_ym32_025_mod)

025deg_jra55v13_iaf_gmredi6


In [8]:
year = temp_ym32_025_mod.year
depth_025_mod = temp_ym32_025_mod.st_ocean
lon_025_mod = temp_ym32_025_mod.xt_ocean

temp_ym32_025_trans_mod = temp_ym32_025_mod
print(np.shape(temp_ym32_025_trans_mod))
temp_ym32_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym32_025')
temp_ym32_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym32_025')
temp_ym32_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym32_025')
for iid, i in enumerate(depth_025_mod):
    for jid, j in enumerate(lon_025_mod):
        temp_ym32_025_slope_mod[iid, jid], _, _, temp_ym32_025_p_value_mod[iid, jid], temp_ym32_025_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym32_025_trans_mod[:, iid, jid])
    print('depth_025_mod ' + str(np.array(i)))
print(temp_ym32_025_slope_mod)

NameError: name 'temp_ym32_025_mod' is not defined

In [9]:
%%time
lonW = 150-360
lonE = 182-360
latL = -32.5
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym32_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=latL, method='nearest').\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym32_01_mod_month)

temp_ym32_01_mod_month.load()
print(temp_ym32_01_mod_month)

xt_ocean = temp_ym32_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym32_01_mod_month = temp_ym32_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym32_01_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym32_01_mod_month = temp_ym32_01_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym32_01_mod_month)

temp_ym32_01_mod = temp_ym32_01_mod_month.groupby('time.year').mean('time')
print(temp_ym32_01_mod)

temp_ym32_01_mod = temp_ym32_01_mod.sel(year=slice(2004,2017))
print(temp_ym32_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 396, st_ocean: 55, xt_ocean: 320)>
dask.array<sub, shape=(396, 55, 320), dtype=float32, chunksize=(1, 7, 220), chunktype=numpy.ndarray>
Coordinates:
    yt_ocean  float64 -32.46
  * xt_ocean  (xt_ocean) float64 -209.9 -209.8 -209.7 ... -178.2 -178.1 -178.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 1.858e+03 2.007e+03
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
<xarray.DataArray 'temp' (time: 396, st_ocean: 55, xt_ocean: 320)>
array([[[       nan,        nan,        nan, ..., 23.55429  ,
         23.560974 , 23.55484  ],
        [       nan,        nan,        nan, ..., 23.49527  ,
         23.500427 , 23.494965 ],
        [       nan,        nan,        nan, ..., 23.474121 ,
         23.479156 , 23.471802 ],
        ...,
        [       nan,        nan,        nan, ...,  2.3882446,
          2.3974304,  2.4033203],
        [       nan,        nan,        nan, ...,  2.2771912,
          2.2853088,  2.2903137],
    

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [10]:
year = temp_ym32_01_mod.year
depth_01_mod = temp_ym32_01_mod.st_ocean
lon_01_mod = temp_ym32_01_mod.xt_ocean

temp_ym32_01_trans_mod = temp_ym32_01_mod
print(np.shape(temp_ym32_01_trans_mod))
temp_ym32_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym32_01')
temp_ym32_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym32_01')
temp_ym32_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym32_01')
for iid, i in enumerate(depth_01_mod):
    for jid, j in enumerate(lon_01_mod):
        temp_ym32_01_slope_mod[iid, jid], _, _, temp_ym32_01_p_value_mod[iid, jid], temp_ym32_01_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym32_01_trans_mod[:, iid, jid])
    print('depth_01_mod ' + str(np.array(i)))
print(temp_ym32_01_slope_mod)

(14, 55, 320)
depth_01_mod -0.5412807653916101
depth_01_mod -1.680734679831433
depth_01_mod -2.939952648914465
depth_01_mod -4.331521485149508
depth_01_mod -5.8693504240540255
depth_01_mod -7.5688099200502155
depth_01_mod -9.446884959648713
depth_01_mod -11.522344392803275
depth_01_mod -13.815927932333222
depth_01_mod -16.350552632935347
depth_01_mod -19.151540835805992
depth_01_mod -22.24687175218868
depth_01_mod -25.667459058670023
depth_01_mod -29.447457087455206
depth_01_mod -33.62459841385048
depth_01_mod -38.24056586677453
depth_01_mod -43.341402211041604
depth_01_mod -48.97796096317406
depth_01_mod -55.20640199365842
depth_01_mod -62.088735720175855
depth_01_mod -69.69341978534455
depth_01_mod -78.09601210263148
depth_01_mod -87.37988400213499
depth_01_mod -97.63699685168024
depth_01_mod -108.96874488209758
depth_01_mod -121.48686590184016
depth_01_mod -135.31441999260392
depth_01_mod -150.5868339523135
depth_01_mod -167.45300594349283
depth_01_mod -186.07646019149857
depth_01_m

In [11]:
temp_ym32_1_mod_xr = xr.DataArray(temp_ym32_1_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym32_1_mod',
                        coords=[depth_1_mod, lon_1_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym32_1_mod_xr)
temp_ym32_1_mod_xr.to_netcdf(output_path + 'temp_ym32_1_mod.nc')

temp_ym32_1_slope_mod_xr = xr.DataArray(temp_ym32_1_slope_mod, name='temp_ym32_1_slope_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_1_slope_mod_xr)
temp_ym32_1_slope_mod_xr.to_netcdf(output_path + 'temp_ym32_1_slope_mod.nc')

temp_ym32_1_std_err_mod_xr = xr.DataArray(temp_ym32_1_std_err_mod, name='temp_ym32_1_std_err_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_1_std_err_mod_xr)
temp_ym32_1_std_err_mod_xr.to_netcdf(output_path + 'temp_ym32_1_std_err_mod.nc')



temp_ym32_025_mod_xr = xr.DataArray(temp_ym32_025_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym32_025_mod',
                        coords=[depth_025_mod, lon_025_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym32_025_mod_xr)
temp_ym32_025_mod_xr.to_netcdf(output_path + 'temp_ym32_025_mod.nc')

temp_ym32_025_slope_mod_xr = xr.DataArray(temp_ym32_025_slope_mod, name='temp_ym32_025_slope_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_025_slope_mod_xr)
temp_ym32_025_slope_mod_xr.to_netcdf(output_path + 'temp_ym32_025_slope_mod.nc')

temp_ym32_025_p_value_mod_xr = xr.DataArray(temp_ym32_025_p_value_mod, name='temp_ym32_025_p_value_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_025_p_value_mod_xr)
temp_ym32_025_p_value_mod_xr.to_netcdf(output_path + 'temp_ym32_025_p_value_mod.nc')

temp_ym32_025_std_err_mod_xr = xr.DataArray(temp_ym32_025_std_err_mod, name='temp_ym32_025_std_err_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_025_std_err_mod_xr)
temp_ym32_025_std_err_mod_xr.to_netcdf(output_path + 'temp_ym32_025_std_err_mod.nc')



temp_ym32_01_mod_xr = xr.DataArray(temp_ym32_01_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym32_01_mod',
                        coords=[depth_01_mod, lon_01_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym32_01_mod_xr)
temp_ym32_01_mod_xr.to_netcdf(output_path + 'temp_ym32_01_mod.nc')

temp_ym32_01_slope_mod_xr = xr.DataArray(temp_ym32_01_slope_mod, name='temp_ym32_01_slope_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_01_slope_mod_xr)
temp_ym32_01_slope_mod_xr.to_netcdf(output_path + 'temp_ym32_01_slope_mod.nc')

temp_ym32_01_p_value_mod_xr = xr.DataArray(temp_ym32_01_p_value_mod, name='temp_ym32_01_p_value_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_01_p_value_mod_xr)
temp_ym32_01_p_value_mod_xr.to_netcdf(output_path + 'temp_ym32_01_p_value_mod.nc')

temp_ym32_01_std_err_mod_xr = xr.DataArray(temp_ym32_01_std_err_mod, name='temp_ym32_01_std_err_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym32_01_std_err_mod_xr)
temp_ym32_01_std_err_mod_xr.to_netcdf(output_path + 'temp_ym32_01_std_err_mod.nc')

<xarray.DataArray 'temp_ym32_1_mod' (st_ocean: 35, xt_ocean: 30, year: 14)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [20.20925283, 20.19412952, 20.15313532, ..., 20.51907652,
         20.61213795, 20.68565039],
        ...,
        [19.45067322, 19.69147716, 19.65782115, ..., 19.62048424,
         20.14006618, 20.00557753],
        [19.36193665, 19.46468634, 19.4957018 , ..., 19.23035908,
         19.78437489, 19.78939249],
        [19.26251277, 19.24312774, 19.33007019, ..., 18.84372514,
         19.53483781, 19.59682272]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [20.19887716, 20.18472985, 20.14355172, ..., 20.50708354,
         20.60243237, 20.67

<xarray.DataArray 'temp_ym32_025_p_value_mod' (st_ocean: 35, xt_ocean: 128)>
array([[       nan,        nan,        nan, ..., 0.08902296, 0.1083003 ,
        0.13491866],
       [       nan,        nan,        nan, ..., 0.09080901, 0.11038302,
        0.13793929],
       [       nan,        nan,        nan, ..., 0.09551349, 0.11609734,
        0.14519353],
       ...,
       [       nan,        nan,        nan, ..., 0.08622026, 0.01692717,
        0.03520203],
       [       nan,        nan,        nan, ..., 0.37981536, 0.19520277,
        0.39369667],
       [       nan,        nan,        nan, ..., 0.32683694, 0.24532121,
        0.41084227]])
Coordinates:
  * st_ocean  (st_ocean) float64 -1.152 -3.649 -6.565 ... -1.854e+03 -2.046e+03
  * xt_ocean  (xt_ocean) float64 150.1 150.4 150.6 150.9 ... 181.4 181.6 181.9
<xarray.DataArray 'temp_ym32_025_std_err_mod' (st_ocean: 35, xt_ocean: 128)>
array([[       nan,        nan,        nan, ..., 0.01684049, 0.01718293,
        0.01731821],
   